# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

## Лабораторная работа 9

In [2]:
import pandas as pd

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [3]:
import nltk
text = 'Pandas is a librarian built on the basis of the functionality of the NumPy library, providing a convenient infrastructure for processing panel data. The main class of Pandas is DataFrame.'
words = list(set(nltk.word_tokenize(text)))
print(words)

['basis', '.', 'panel', 'functionality', 'librarian', 'library', 'providing', 'the', 'a', 'convenient', 'infrastructure', 'Pandas', 'DataFrame', 'main', 'processing', 'for', 'is', 'NumPy', ',', 'built', 'class', 'The', 'of', 'data', 'on']


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [4]:
import random
from nltk.metrics import *
lst = [0] * 5
for i in range(5):
    lst[i] = random.sample(words, 2)
    print(f'Пара: {lst[i]}, расстояние редактирования: {edit_distance(lst[i][0], lst[i][1])}')

Пара: ['a', 'panel'], расстояние редактирования: 4
Пара: ['providing', 'librarian'], расстояние редактирования: 8
Пара: [',', 'library'], расстояние редактирования: 7
Пара: ['is', 'a'], расстояние редактирования: 2
Пара: ['basis', 'library'], расстояние редактирования: 6


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [5]:
def find_words(word, k):
    dct = {w: edit_distance(w, word) for w in words}
    dct = sorted(dct, key = dct.get)[:k]
    return dct
print(find_words('class', 5))

['class', 'basis', 'a', 'main', 'is']


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [6]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tyute\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
from nltk.stem import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer(language='english')
df = pd.DataFrame({'stemmed_word': [stemmer.stem(word) for word in words], 'normalized_word': [lemmatizer.lemmatize(word) for word in words]}, index=words)
df

,stemmed_word,normalized_word
basis,basi,basis
.,.,.
panel,panel,panel
functionality,function,functionality
librarian,librarian,librarian
library,librari,library
providing,provid,providing
the,the,the
a,a,a
convenient,conveni,convenient


In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tyute\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [12]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

text = 'Pandas is a librarian built on the basis of the functionality of the NumPy library, providing a convenient infrastructure for processing panel data. The main class of Pandas is DataFrame.'

tokens = word_tokenize(text.lower())
english_stopwords = stopwords.words('english')
tokens_without_stopwords = [word for word in tokens if word not in english_stopwords]

print(f"Доля стоп-слов в тексте: {len(tokens) / len(english_stopwords):.2%}")
print(f"Топ-10 слов до удаления стоп-слов: {[word for word, count in Counter(tokens).most_common(10)]}")
print(f"Топ-10 после удаления стоп-слов: {[word for word, count in Counter(tokens_without_stopwords).most_common(10)]}")


Доля стоп-слов в тексте: 18.44%
Топ-10 слов до удаления стоп-слов: ['the', 'of', 'pandas', 'is', 'a', '.', 'librarian', 'built', 'on', 'basis']
Топ-10 после удаления стоп-слов: ['pandas', '.', 'librarian', 'built', 'basis', 'functionality', 'numpy', 'library', ',', 'providing']


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer = TfidfVectorizer()
recipes = pd.read_csv('recipes_sample.csv', delimiter=',')
recipes = recipes.head()
vectors = vectorizer.fit_transform(recipes['description'])

for i in range(vectors.shape[0]):
    print(f"Рецепт {i+1}: {vectors[i].toarray()}")


Рецепт 1: [[0.         0.         0.         0.         0.         0.09912595
  0.06921947 0.         0.         0.32913396 0.         0.
  0.         0.09912595 0.         0.09912595 0.12286406 0.
  0.         0.12286406 0.         0.         0.12286406 0.12286406
  0.         0.         0.         0.19825189 0.         0.
  0.12286406 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.12286406 0.         0.         0.08228349
  0.         0.         0.08228349 0.         0.         0.
  0.12286406 0.         0.12286406 0.09912595 0.09912595 0.
  0.09912595 0.16456698 0.12286406 0.12286406 0.         0.
  0.12286406 0.         0.         0.         0.         0.
  0.         0.         0.         0.12286406 0.         0.
  0.         0.         0.12286406 0.         0.         0.
  0.         0.12286406 0.         0.         0.         0.
  0.12286406 0.         0.         0.         0.         0.1228640

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [14]:
from scipy.spatial import distance
import numpy
import pandas as pd
lst =[]
for el in recipes['description']:
    lst.append(word_tokenize(el))
vectors  = vectorizer.fit_transform(recipes['description']).toarray()
for index_1 in range(len(lst)):
    for index_2 in range(index_1 + 1, len(lst)):
        print(distance.cosine(vectors[index_1], vectors[index_2]))

0.9725198666999536
0.8944228631111404
0.8408701219963103
0.8093525908759086
1.0
0.8551689750098787
0.9214042717289879
0.9640049285228507
0.9529584291886545
0.8788072331736545
